In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
model_name = "sapienzanlp/Minerva-3B-base-v1.0"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
dataset = load_dataset("anakin87/fine-instructions-ita-70k")

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['conversations', 'id', 'quality'],
        num_rows: 69890
    })
})

In [6]:
# Rename 'conversations' → 'messages'
def rename_conversations(example):
    example["messages"] = example["conversations"]
    return example

processed_dataset = dataset["train"].map(rename_conversations)


In [7]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # You might need to adjust for your model
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


In [8]:
model = get_peft_model(model, lora_config)

In [13]:
training_args = TrainingArguments(
    output_dir="./lora-sft-out",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    fp16=False,                 # Full precision
    bf16=False,                 # Use bf16=True if you have Ampere+ GPUs and want mixed precision
    save_total_limit=2,
)


In [ ]:
my_chat_template = """
{% set loop_messages = messages %}
{% for message in loop_messages %}
    {% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\\n\\n' + message['content'] | trim + '<|eot_id|>' %}
    {% if loop.index0 == 0 %}
        {% set content = bos_token + content %}
    {% endif %}
    {{ content }}
{% endfor %}
{% if add_generation_prompt %}
    {{ '<|start_header_id|>assistant<|end_header_id|>\\n\\n' }}
{% endif %}
"""

tokenizer.chat_template = my_chat_template

In [15]:
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    args=training_args,
    train_dataset=processed_dataset,
)

OutOfMemoryError: CUDA out of memory. Tried to allocate 88.00 MiB. GPU 0 has a total capacity of 23.55 GiB of which 4.44 MiB is free. Process 625194 has 11.43 GiB memory in use. Process 625809 has 10.97 GiB memory in use. Including non-PyTorch memory, this process has 1.09 GiB memory in use. Of the allocated memory 714.04 MiB is allocated by PyTorch, and 15.96 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)